# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
#import libs
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
        Trainer, TrainingArguments, DataCollatorWithPadding

In [2]:
#load gpt2 model and configure tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token


model_gpt = AutoModelForSequenceClassification.from_pretrained(
    "openai-community/gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)
model_gpt.config.pad_token_id = tokenizer.pad_token_id

for param in model_gpt.base_model.parameters():
    param.requires_grad = False

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# examine model structure
print(model_gpt)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [4]:
!pip install -U datasets

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from datasets import load_dataset
#load imdb dataset
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# tokenize all the examples
def tokenize_batch(batch):
    return tokenizer(batch["text"],padding="max_length", truncation=True)
tokenized_dataset = dataset.map(tokenize_batch, batched=True)

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [14]:
#split into train and test set and sub sample 500 for faster training and inferencing
train = tokenized_dataset["train"].shuffle(seed=42).select(range(500))
test = tokenized_dataset["test"].shuffle(seed=42).select(range(500))

In [7]:
#install scikit for accuracy evaluation
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 42.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 37.2 MB/s eta 0:00:00


In [18]:
#train model head
import numpy as np
from sklearn.metrics import accuracy_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

training_args = TrainingArguments(
    output_dir="./test_trainer",
    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=1e-3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

gpt_trainer = Trainer(
    model=model_gpt,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    #data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

gpt_eval = gpt_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.678948,0.596000


Checkpoint destination directory ./test_trainer/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [19]:
gpt_trainer.save_model("base_gpt_model")

In [15]:
import pandas as pd

df = pd.DataFrame(test)
df = df[["text", "label"]]

# Replace <br /> tags in the text with spaces
df["text"] = df["text"].str.replace("<br />", " ")

# Add the model predictions to the dataframe
predictions = gpt_trainer.predict(test)
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head(4)

,text,label,predicted_label
0,When I unsuspectedly rented A Thousand Acres...,1,1
1,This is the latest entry in the long series of...,1,1
2,This movie was so frustrating. Everything seem...,0,0
3,"I was truly and wonderfully surprised at ""O' B...",1,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [20]:
model_base_trained = AutoModelForSequenceClassification.from_pretrained("base_gpt_model")

In [21]:
from peft import LoraConfig, get_peft_model, TaskType


lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model_base_trained, lora_config)

In [22]:
trainer_lora = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./loraModel",
        learning_rate=1e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.343102,0.894000
2,No log,0.393842,0.890000


Checkpoint destination directory ./loraModel/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./loraModel/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=250, training_loss=0.49543072509765623, metrics={'train_runtime': 314.5536, 'train_samples_per_second': 3.179, 'train_steps_per_second': 0.795, 'total_flos': 527585771520000.0, 'train_loss': 0.49543072509765623, 'epoch': 2.0})

In [23]:
trainer_lora.evaluate()

{'eval_loss': 0.3431020975112915,
 'eval_accuracy': 0.894,
 'eval_runtime': 46.1087,
 'eval_samples_per_second': 10.844,
 'eval_steps_per_second': 2.711,
 'epoch': 2.0}

In [24]:
lora_model.print_trainable_parameters()

trainable params: 814,080 || all params: 125,253,888 || trainable%: 0.6499438963523432


In [25]:
lora_model.save_pretrained("gpt2_lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt2_lora")

tokenizer = AutoTokenizer.from_pretrained("gpt2")

inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = lora_model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer.batch_decode(outputs))

TypeError: The current model class (GPT2ForSequenceClassification) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'GPT2LMHeadModel'}